In [ ]:
# A notebook for stepping through an episode and visualizing the
# imitation agent's behaviour against the original at every step.

In [ ]:
import copy
import json

from kaggle_environments import make
from kaggle_environments.envs.halite.helpers import (
    Board,
    board_agent,
    ShipAction,
    ShipyardAction,
)

In [ ]:
def visualize_agent_action(game, agent, step, team_name):
    """Visualize the step that an agent *would* have taken
    given a state from a real game.
    """
    # Determine whether team of interest is player 0 or 1.
    # Raises exception if neither team name matches.
    team_idx = game["info"]["TeamNames"].index(team_name)
    
    config = game["configuration"]
    
    # Create environment with steps up to the current step of interest.
    tmp_env = make('halite', configuration=config, steps=imitation_game['steps'][:step+1])
    state = copy.deepcopy(tmp_env.state[0])
    # Get actions that were taken from this state in the original game.
    next_actions = [
        imitation_game['steps'][step+1][0]['action'],
        imitation_game['steps'][step+1][1]['action'],
    ]
    # Wipe actions for one user - we are going to use an agent instead.
    next_actions[team_idx] = {}
    # Set current player to be the player of interest.
    state["observation"]["player"] = team_idx

    board = Board(state["observation"], config, next_actions)
    agent(board) # agent should update current_player next_actions in-place.
    
    tmp_env.step([board.players[0].next_actions, board.players[1].next_actions])
    tmp_env.render(mode="ipython", width=500, height=350, autoplay=False)

In [ ]:
# Dummy agent.
def move_ships_north_agent(board):
    current_player = board.current_player
    for ship in current_player.ships:
        ship.next_action = ShipAction.NORTH

In [ ]:
# Load game of interest.
game_path = "../data/submission_17327012_episodes/3602885.json"
with open(game_path) as f:
    game = json.loads(json.load(f)["replay"])
imitation_game = copy.deepcopy(game)

In [ ]:
# Render original game.
env = make('halite', configuration=game['configuration'], steps=game['steps'])
env.render(mode="ipython", width=500, height=400, autoplay=False)
# Render imitation step.
step = 219
print(f"Step {step+1} -> {step+2}") # Step numbering in viewer starts at 1.
visualize_agent_action(imitation_game, move_ships_north_agent, step=219, team_name="Stanley Zheng")